In [1]:
import numpy as np
import pandas as pd

In [2]:
fp = pd.read_csv('/Users/dayeong/Downloads/all_intradata_dy.csv')

In [3]:
fp.head()

,time,value,date
0,0:00:01,83,2022-04-12
1,0:00:06,86,2022-04-12
2,0:00:11,85,2022-04-12
3,0:00:16,83,2022-04-12
4,0:00:21,84,2022-04-12


In [4]:
fp.tail()

,time,value,date
12649,23:58:22,70,2022-04-12
12650,23:58:27,68,2022-04-12
12651,23:58:42,67,2022-04-12
12652,23:58:52,68,2022-04-12
12653,23:58:57,70,2022-04-12


In [5]:
fp_test = fp[(fp['time']>'14:00:00') & (fp['time'] < '14:05:00')]

In [6]:
fp_test.columns

Index(['time', 'value', 'date'], dtype='object')

In [7]:
fp_test.mean()

value    85.027778
dtype: float64

In [8]:
fp_total = fp[(fp['time']>'00:00:00') & (fp['time'] < '23:59:59')]

In [9]:
fp_total.mean()

value    91.21322
dtype: float64

In [10]:
nn_intervals_test = fp_test['value']

In [11]:
def get_time_domain_features(nn_intervals_test) -> dict:

    diff_nni = np.diff(nn_intervals_test)
    length_int = len(nn_intervals_test)

    # Basic statistics
    mean_nni = np.mean(nn_intervals_test)
    median_nni = np.median(nn_intervals_test)
    range_nni = max(nn_intervals_test) - min(nn_intervals_test)

    sdsd = np.std(diff_nni)
    rmssd = np.sqrt(np.mean(diff_nni ** 2))

    nni_50 = sum(np.abs(diff_nni) > 50)
    pnni_50 = 100 * nni_50 / length_int
    nni_20 = sum(np.abs(diff_nni) > 20)
    pnni_20 = 100 * nni_20 / length_int

    # Feature found on github and not in documentation
    cvsd = rmssd / mean_nni

    # Features only for long term recordings
    sdnn = np.std(nn_intervals_test, ddof=1)  # ddof = 1 : unbiased estimator => divide std by n-1
    cvnni = sdnn / mean_nni

    # Heart Rate equivalent features
    heart_rate_list = 60000/nn_intervals_test
    mean_hr = np.mean(heart_rate_list)
    min_hr = min(heart_rate_list)
    max_hr = max(heart_rate_list)
    std_hr = np.std(heart_rate_list)

    time_domain_features = {
        'mean_nni': mean_nni,
        'sdnn': sdnn,
        'sdsd': sdsd,
        'nni_50': nni_50,
        'pnni_50': pnni_50,
        'nni_20': nni_20,
        'pnni_20': pnni_20,
        'rmssd': rmssd,
        'median_nni': median_nni,
        'range_nni': range_nni,
        'cvsd': cvsd,
        'cvnni': cvnni,
        'mean_hr': mean_hr,
        "max_hr": max_hr,
        "min_hr": min_hr,
        "std_hr": std_hr,
    }

    return time_domain_features

In [12]:
get_time_domain_features(nn_intervals_test) # 5분

{'mean_nni': 85.02777777777777,
 'sdnn': 2.157857550059597,
 'sdsd': 1.124858267715973,
 'nni_50': 0,
 'pnni_50': 0.0,
 'nni_20': 0,
 'pnni_20': 0.0,
 'rmssd': 1.1338934190276817,
 'median_nni': 85.0,
 'range_nni': 9,
 'cvsd': 0.01333556454916581,
 'cvnni': 0.02537826586153071,
 'mean_hr': 706.0855571760259,
 'max_hr': 731.7073170731708,
 'min_hr': 659.3406593406594,
 'std_hr': 17.345850732472485}

In [13]:
nn_intervals_total = fp_total['value']

In [18]:
def get_time_domain_features(nn_intervals_total) -> dict:

    diff_nni = np.diff(nn_intervals_total)
    length_int = len(nn_intervals_total)

    # Basic statistics
    mean_nni = np.mean(nn_intervals_total)
    median_nni = np.median(nn_intervals_total)
    range_nni = max(nn_intervals_total) - min(nn_intervals_total)

    sdsd = np.std(diff_nni)
    rmssd = np.sqrt(np.mean(diff_nni ** 2))

    nni_50 = sum(np.abs(diff_nni) > 50)
    pnni_50 = 100 * nni_50 / length_int
    nni_20 = sum(np.abs(diff_nni) > 20)
    pnni_20 = 100 * nni_20 / length_int

    # Feature found on github and not in documentation
    cvsd = rmssd / mean_nni

    # Features only for long term recordings
    sdnn = np.std(nn_intervals_total, ddof=1)  # ddof(자유도) = 1 : unbiased estimator => divide std by n-1
    cvnni = sdnn / mean_nni

    # Heart Rate equivalent features
    
    # NN 간격과 심박동수의 관계는 심박동수 = 60/NN 간격이다. 
    heart_rate_list = nn_intervals_total
    mean_hr = np.mean(heart_rate_list)
    min_hr = min(heart_rate_list)
    max_hr = max(heart_rate_list)
    std_hr = np.std(heart_rate_list)
    
    # 스트레스 레벨
    stress_level = 2*mean_hr-sdnn

    time_domain_features = {
        'mean_nni': mean_nni,
        'sdnn': sdnn,
        'sdsd': sdsd,
        'nni_50': nni_50,
        'pnni_50': pnni_50,
        'nni_20': nni_20,
        'pnni_20': pnni_20,
        'rmssd': rmssd,
        'median_nni': median_nni,
        'range_nni': range_nni,
        'cvsd': cvsd,
        'cvnni': cvnni,
        'mean_hr': mean_hr,
        "max_hr": max_hr,
        "min_hr": min_hr,
        "std_hr": std_hr,
        "stress_level" : stress_level
        
    }

    return time_domain_features

In [19]:
get_time_domain_features(nn_intervals_total)

{'mean_nni': 91.21322033898305,
 'sdnn': 12.978491962199525,
 'sdsd': 2.0075848069007507,
 'nni_50': 0,
 'pnni_50': 0.0,
 'nni_20': 0,
 'pnni_20': 0.0,
 'rmssd': 2.007585344420419,
 'median_nni': 89.0,
 'range_nni': 86,
 'cvsd': 0.022009806659160455,
 'cvnni': 0.1422873999401239,
 'mean_hr': 91.21322033898305,
 'max_hr': 148,
 'min_hr': 62,
 'std_hr': 12.977758693351673,
 'stress_level': 169.4479487157666}

In [26]:
nn_intervals_total.max()

148

In [1]:
# 만성스트레스 지수 = 2M -(Tsdnn + Thrv-index)/2
# 2M의 M = 각각의 평균이 되는 평균
# Tsdnn = 스트레스 저항도(SDNN)의 표준지표 Ti
# Thrv-index = 심기능 활성도(HRV-INDEX)의 표준지표 Tj

In [3]:
# Triangular index = 밀도분포의 적분 
# HRV triangular index = (total number of NN interval)/(number of NN intervals in the modal bin)

In [ ]:
2*